In [61]:
import datetime
import pandas as pd
import numpy as np
from sklearn.metrics import jaccard_score
import operator

In [43]:
dataTr = pd.read_csv('dataTr.csv')
dataTe = pd.read_csv('dataTe.csv')

In [44]:
# currently, dataTr and dataTe are the same
dataTe.columns =['Index', 'Rank', 'Name', 'City', 'State', 'PubOrPriv',
                 'UndergradPop', 'StudentPop', 'NetPrice', 'AverageGrantAid', 'TotalAnnualCost', 'AlumniSalary', 'AcceptanceRate', 'SATLower', 'SATUpper', 'ACTLower', 'ACTUpper', 'Website']
dataTr.columns =['Index', 'Rank', 'Name', 'City', 'State', 'PubOrPriv',
                 'UndergradPop', 'StudentPop', 'NetPrice', 'AverageGrantAid', 'TotalAnnualCost', 'AlumniSalary', 'AcceptanceRate', 'SATLower', 'SATUpper', 'ACTLower', 'ACTUpper', 'Website']
dataTr.head()

,Index,Rank,Name,City,State,PubOrPriv,UndergradPop,StudentPop,NetPrice,AverageGrantAid,TotalAnnualCost,AlumniSalary,AcceptanceRate,SATLower,SATUpper,ACTLower,ACTUpper,Website
0,0,1.0,Harvard University,Cambridge,MA,Private,13844.0,31120.0,14327.0,49870.0,69600.0,146800.0,5.0,1460.0,1590.0,32.0,35.0,www.harvard.edu
1,1,2.0,Stanford University,Stanford,CA,Private,8402.0,17534.0,13261.0,50134.0,69109.0,145200.0,5.0,1390.0,1540.0,32.0,35.0,www.stanford.edu
2,2,3.0,Yale University,New Haven,CT,Private,6483.0,12974.0,18627.0,50897.0,71290.0,138300.0,7.0,1460.0,1580.0,32.0,35.0,www.yale.edu
3,3,4.0,Massachusetts Institute of Technology,Cambridge,MA,Private,4680.0,11466.0,20771.0,43248.0,67430.0,155200.0,7.0,1490.0,1570.0,33.0,35.0,web.mit.edu
4,4,5.0,Princeton University,Princeton,NJ,Private,5659.0,8273.0,9327.0,48088.0,66150.0,139400.0,6.0,1430.0,1570.0,31.0,35.0,www.princeton.edu


In [45]:
# Size
# small: < 5,000
# medium: 5,000 - 15,000
# large: > 15,000

# Location
Northeast = ['PA', 'NY', 'NJ', 'ME', 'NH', 'VT', 'MA', 'CT', 'RI']
Midwest = ['ND', 'SD', 'NE', 'KS', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'MI', 'OH']
South = ['OK', 'TX', 'AR', 'LA', 'KY', 'TN', 'MS', 'AL', 'WV', 'MD', 'VA', 'DC', 'NC', 'SC', 'GA', 'AL', 'FL', 'DE']
West = ['WA', 'OR', 'CA', 'NV', 'ID', 'MT', 'WY', 'UT', 'AZ', 'NM', 'CO']

for index, row in dataTr.iterrows():
    size = dataTr.loc[index]['UndergradPop']
    if (size < 5000):
        dataTr.loc[index,'Size'] = 'Small'
    elif (size >= 5000 and size < 15000):
        dataTr.loc[index,'Size'] = 'Medium'
    else:
        dataTr.loc[index,'Size'] = 'Large'
    
    state = dataTr.loc[index]['State']
    if (state in Northeast):
        dataTr.loc[index,'Location'] = 'Northeast'
    elif (state in Midwest):
        dataTr.loc[index,'Location'] = 'Midwest'
    elif (state in South):
        dataTr.loc[index,'Location'] = 'South'
    else:
        dataTr.loc[index,'Location'] = 'West'
    
    satAvg = (dataTr.loc[index]['SATLower'] + dataTr.loc[index]['SATUpper'])/2
    dataTr.loc[index,'satAvg'] = satAvg

    actAvg = (dataTr.loc[index]['ACTLower'] + dataTr.loc[index]['ACTUpper'])/2
    dataTr.loc[index,'actAvg'] = actAvg

dataTr

,Index,Rank,Name,City,State,PubOrPriv,UndergradPop,StudentPop,NetPrice,AverageGrantAid,...,AcceptanceRate,SATLower,SATUpper,ACTLower,ACTUpper,Website,Size,Location,satAvg,actAvg
0,0,1.0,Harvard University,Cambridge,MA,Private,13844.0,31120.0,14327.0,49870.0,...,5.0,1460.0,1590.0,32.0,35.0,www.harvard.edu,Medium,Northeast,1525.0,33.5
1,1,2.0,Stanford University,Stanford,CA,Private,8402.0,17534.0,13261.0,50134.0,...,5.0,1390.0,1540.0,32.0,35.0,www.stanford.edu,Medium,West,1465.0,33.5
2,2,3.0,Yale University,New Haven,CT,Private,6483.0,12974.0,18627.0,50897.0,...,7.0,1460.0,1580.0,32.0,35.0,www.yale.edu,Medium,Northeast,1520.0,33.5
3,3,4.0,Massachusetts Institute of Technology,Cambridge,MA,Private,4680.0,11466.0,20771.0,43248.0,...,7.0,1490.0,1570.0,33.0,35.0,web.mit.edu,Small,Northeast,1530.0,34.0
4,4,5.0,Princeton University,Princeton,NJ,Private,5659.0,8273.0,9327.0,48088.0,...,6.0,1430.0,1570.0,31.0,35.0,www.princeton.edu,Medium,Northeast,1500.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,518,645.0,Morehouse College,Atlanta,GA,Private,2202.0,2219.0,34928.0,17318.0,...,74.0,950.0,1160.0,18.0,23.0,https://www.morehouse.edu,Small,South,1055.0,20.5
519,519,646.0,New Mexico State University,Las Cruces,NM,Public,13379.0,14432.0,8625.0,9582.0,...,64.0,910.0,1160.0,18.0,23.0,www.nmsu.edu,Medium,West,1035.0,20.5
520,520,647.0,Indiana State University,Terre Haute,IN,Public,13626.0,13763.0,13012.0,9297.0,...,85.0,900.0,1110.0,17.0,23.0,www.indstate.edu,Medium,Midwest,1005.0,20.0
521,521,648.0,Emory and Henry College,Emory,VA,Private,1094.0,1226.0,19340.0,27155.0,...,72.0,988.0,1170.0,19.0,25.0,www.ehc.edu,Small,South,1079.0,22.0


In [46]:
# print(dataTr['Size'].value_counts())
# print(dataTr['Location'].value_counts())
# print(dataTr['satAvg'].value_counts())
# print(dataTr['actAvg'].value_counts())

In [77]:
similarity = {}
dataTrDroppedCols = dataTr[['satAvg', 'actAvg']].copy()

# satAvg, actAvg jaccard sim
y_true = [1110, 30]

# jaccard_score(y_true, row)

def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return round(float(intersection) / union, 2)

for index, row in dataTrDroppedCols.iterrows():
    jacScore = jaccard(y_true, row)
    collegeName = dataTr.loc[index]['Name']
    similarity[collegeName] = jacScore
    
sorted_sim = sorted(similarity.items(),key=operator.itemgetter(1),reverse=True)

In [78]:
sorted_sim

[('Santa Clara University', 0.33),
 ('University of Florida', 0.33),
 ('University of California, San Diego', 0.33),
 ('Whitman College', 0.33),
 ('University of Miami (FL)', 0.33),
 ('Southern Methodist University', 0.33),
 ('Colorado School of Mines', 0.33),
 ('Trinity College (CT)', 0.33),
 ('Rensselaer Polytechnic Institute', 0.33),
 ('Moravian College', 0.33),
 ('Utah State University', 0.33),
 ('Texas State University', 0.33),
 ('Bowling Green State University', 0.33),
 ('Harvard University', 0.0),
 ('Stanford University', 0.0),
 ('Yale University', 0.0),
 ('Massachusetts Institute of Technology', 0.0),
 ('Princeton University', 0.0),
 ('University of Pennsylvania', 0.0),
 ('Brown University', 0.0),
 ('California Institute of Technology', 0.0),
 ('Duke University', 0.0),
 ('Dartmouth College', 0.0),
 ('Cornell University', 0.0),
 ('Pomona College', 0.0),
 ('University of California, Berkeley', 0.0),
 ('Columbia University', 0.0),
 ('Georgetown University', 0.0),
 ('University of 